# Symmetrie
Um das Caching noch effektiver zu gestalten, sollen neben Transpositionen auch Symmetrien erkannt werden. In diesem Kapitel werden alle Funktionen, die für die Symmetrieerkennung nötig sind, vorgestellt und implementiert.

Zunächst werden Hilfsfunktion definiert, die auf ein gegebenen Spielbrettern (`boards`) eine bestimmte Symmetrie anwenden alle resultierenden Spielbretter in einer Menge (`Set`) zurück geben. Eine weitere Hilfsfunktion wendet auf Zustände (`states`) eine weitere Symmetrie an.
Schlussendlich werden alle Symmetrien nacheinander angewandte, damit auch zusammengesetzte Symetrien wie beispielsweise `Rotation um 90°` dann `Spiegelung an der horizontalen Achse` errechnet werden.

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [ ]:
%run ./nmm-game-utils.ipynb

## Rotation
Ein Spielbrett kann um 90°, 180° oder 270° gedreht werden, die resultierenden Spielbretter sind rotationssymmetrisch.

Die Eingabe besteht aus einem einzelnen Spielbrett (`board`), die Ausgabe ist eine Menge, die maximal aus drei Spielbrettern besteht, die rotationssymmetrisch zu der Eingabe sind.
Berechnet wird die Ausgabe indem alle Ringe um `k ∈ {2, 4, 6}` Zellen rotiert werden. Durch Aneinanderreihung der letzten `8-k` Zellen und der ersten `k` Zellen kommt die Rotation zustande.

In [ ]:
def symmetryRotation(boards):
    return {
        tuple(
            board[ring][rotation:] + board[ring][:rotation]
            for ring in range(3)
        )
        for rotation in range(2, 6+1, 2)
        for board in boards
    }

## Spiegelung
Bei den Spiegelungen wird an vier Achsen gespiegelt:
* die *horizontale* und *vertikale* Achse, sowie
* die Diagonale von oben links nach unten rechts (*negative Diagonale*) und die Diagonale von unten links nach oben rechts (*positive Diagnonale*).

Diese Spiegelungen können einzelnt pro Ring vorgenommen werden, da der äußere Ring bleibt nach der Spiegelung weiterhin der äußere Ring. Gleiches gilt für die anderen Ringe. Alle Spiegelungen lassen sich durch eine Invertierung der Ringe und eine Rotation von `k ∈ {0, 2, 4, 6}` darstellen.

In [ ]:
def symmetryHorizontal(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-(cell+2))%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

In [ ]:
def symmetryVertical(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-(cell+6))%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

In [ ]:
def symmetryDiagonalPositive(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-(cell+4))%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

In [ ]:
def symmetryDiagnoalNegative(boards):
    return {
        tuple(
            tuple(
                board[ring][(8-cell)%8]
                for cell in range(8)
            )
            for ring in range(3)
        )
        for board in boards
    }

## Ring-Tausch
Da der innere und der äußere Ring über symmetrische Kanten mit dem mittleren Ring verbunden ist, können der äußere und der innere Ringe getauscht werden. Dies funktioniert indem rückwärts über die Ringe iteriert wird.

In [ ]:
def symmetryRing(boards):
    return {
        tuple(
            board[ring]
            for ring in reversed(range(3))
        )
        for board in boards
    }

## Spieler-Tausch
Die Spieler können ebenfalls vertauscht werden. Wichtig hierbei ist allerdings, dass auch die Steine im `Stash` getauscht werden. Deshalb muss benötigt diese Hilfsfunktion den gesamten Zustand und nicht nur ein einzelnes Spielbrett.

Durch einfaches tauschen des `Stash` und Iteration über alle Zellen, bei der jeder Stein mit dem gegnerischen Stein ausgetauscht wird, lässt sich diese Funktion implementieren.

In [ ]:
def symmetryPlayer(states):
    return {
        (
            (stash[1], stash[0]),
            tuple(
                tuple(
                    ' ' if cell == ' ' else opponent(cell)
                    for cell in ring
                )
                for ring in board
            )
        )
        for stash, board in states
    }

## Zusammenführung
Damit alle möglichen Symmetrien gefunden werden, wird jede Hilfsfunktion einzelnt auf alle vorherigen Spielbretter (`boards`) oder Zustände (`states`) angewandt. Dadurch sind auch zusammengesetzte Symmetrien wie beispielsweise `Rotation um 90°` dann `Spiegelung an der horizontalen Achse` möglich. Mit Hilfe einer Menge wird sichergestellt, dass keine Duplikate zurück gegeben werden.

Da nicht alle Symmetrie-Hilfsfunktionen den gesamten Zustand brauchen, werden zunächst einmal alle Funktionen angewandt, die nur das Spielbrett benötigen. Daraufhin wird die letzte Symmetrie angewandt, die Zustände als Eingabe benötigt.

In [ ]:
def findSymmetries(state):
    stash, board = state
    
    boards = { board }
    boards |= symmetryRotation(boards)
    boards |= symmetryHorizontal(boards)
    boards |= symmetryVertical(boards)
    boards |= symmetryDiagonalPositive(boards)
    boards |= symmetryDiagnoalNegative(boards)
    boards |= symmetryRing(boards)
    
    states = {
        (stash, board)
        for board in boards
    }
    states |= symmetryPlayer(states)
    return states